Steepest Decent 和 Newton's Method是两个优化算法

* Line Search Algorithm的基本思想是，对于某个函数，在某一点x_k重，如果提督f'(x_k) != 0，则可以构造一条射线x_k + ap_k，然后在这条射线上寻找合适的步长a，使得函数值下降

关注:
* Decent Direction 梯度下降方向
* Step Length Selection 步长选择

Wolfe Condition: 一种Step Length Selection策略，包含2个条件：
* Sufficient Decrease Condition, Armijo Condition
    * 要求函数值的下降幅度至少和步长线性相关
* Curvature Condition
    * 只有Sufficient Decrease Condition可能导致步长过小，影响优化速度。

Different Curvature Conditions:
* Strong Curvature Condition -> Strong Wolfe Condition
* Curvature Condition -> Wolfe Condition


Backtracking Linesearch
* directly apply sufficient decrease condition, no checking curvature condition
* Step Direction Selection Algoeirhms:
    * First-order Taylor Expansion -> Steepest Decent
    * Second-order Taylor Expansion -> Newton's Method

Steepest Decent's limitation:
* 当一阶线性模型没有下界的时候，p随着梯度递减，无法求得最优步长方向；此时直接取最速下降的方向


Newton's Method's limitation:
* Hessian is not positive definite





In [2]:
import numpy as np
import time

def backtracking_line_search(f, grad_f, x, p, alpha=1.0, rho=0.5, c1=1e-4):
    """
        Backtracking Line Search
    :param f: target function
    :param grad_f: gradient function of the target function
    :param x: initial val
    :param p: search direction
    :param alpha: initial step, default=1.0
    :param rho: step shrink ratio, default by 0.5
    :param c1: Wolfe condition param, default by 1e-4
    :return: step selected alpha
    """
    while f(x + alpha * p) > f(x) + c1 * alpha * np.dot(grad_f(x), p):
        alpha *= rho
    return alpha

def steepest_descent(f, grad_f, x0, tol=1e-6, max_iter=1000):
    """
       Steepest Descent
    :param f: target function
    :param grad_f: gradient function of the target function
    :param x0: initial val
    :param tol: tolerance point
    :param max_iter: maximum iteration
    :return: opt point and search history
    """
    x = x0
    history = [x]
    for _ in range(max_iter): # stopping criteria: max iteration number
        grad = grad_f(x)
        if np.linalg.norm(grad) < tol:
            break
        p = -grad  # 最陡下降方向
        alpha = backtracking_line_search(f, grad_f, x, p, alpha=0.1)
        x = x + alpha * p
        history.append(x)
    return x, history

def newton_method(f, grad_f, hessian_f, x0, tol=1e-6, max_iter=100):
    """
        Newton's Method
    :param f: target function
    :param grad_f: gradient function of the target function
    :param hessian_f: the Hessian matrix of the target function
    :param x0: initial val
    :param tol: tolerance point
    :param max_iter: maximum iteration
    :return: opt point and search history
    """
    x = x0
    history = [x]
    for _ in range(max_iter): # stopping criteria: max iteration number
        grad = grad_f(x)
        if np.linalg.norm(grad) < tol: # stopping cirteria: when gradient is small enough, break.
            break
        hess = hessian_f(x)
        try:
            p = -np.linalg.solve(hess, grad)  # calculate the direction
        except np.linalg.LinAlgError:
            p = -grad  # if Hessian matrix is not nvertible, then fall back to gradient descent.
        alpha = backtracking_line_search(f, grad_f, x, p)
        x = x + alpha * p
        history.append(x)
    return x, history

def benchmark(method, f, grad_f, hessian_f, x0):
    """
    Benchmark Protocol
    :param method: steepest_descent or newton_method
    :param f: target function
    :param grad_f: gradient of the target function
    :param hessian_f: Hessian of the target function, only used by Newton's
    :param x0: initial point
    """
    start_time = time.time()
    if method == "steepest_descent":
        x_opt, history = steepest_descent(f, grad_f, x0)
    elif method == "newton":
        x_opt, history = newton_method(f, grad_f, hessian_f, x0)
    else:
        raise ValueError("Unsupported method")
    end_time = time.time()
    return {
        "method": method,
        "iterations": len(history),
        "final_x": x_opt,
        "final_f": f(x_opt),
        "runtime": end_time - start_time,
    }


def f(x):
    return (x[0] - 1) ** 2 + (x[1] - 2) ** 2

def grad_f(x):
    return np.array([2 * (x[0] - 1), 2 * (x[1] - 2)])

def hessian_f(x):
    return np.array([[2, 0], [0, 2]])

## benchmark
x0 = np.array([0.0, 0.0])
result_sd = benchmark("steepest_descent", f, grad_f, None, x0)
result_newton = benchmark("newton", f, grad_f, hessian_f, x0)

print("Steepest Descent:", result_sd)
print("Newton's Method:", result_newton)


Steepest Descent: {'method': 'steepest_descent', 'iterations': 70, 'final_x': array([0.99999979, 1.99999959]), 'final_f': 2.1153791005922307e-13, 'runtime': 0.00489497184753418}
Newton's Method: {'method': 'newton', 'iterations': 2, 'final_x': array([1., 2.]), 'final_f': 0.0, 'runtime': 0.002532958984375}
